In [7]:
import numpy as np
import tqdm

grids = np.load("/home/tatp/Desktop/grids.npy")

base_scale = 0.033 # you can tune this
output_ply = "/home/tatp/Desktop/points_thresh_supersplat.ply"

positions = grids[:, :3].astype(np.float32)
N = int(positions.shape[0])

def RGB2SH(rgb):
    C0 = 0.28209479177387814
    return (rgb - 0.5).astype(np.float32) / C0
_rgb = np.random.random((N, 3)).astype(np.float32)
shs = RGB2SH(_rgb)

opa_ = np.ones(N, dtype=np.float32)
opacities = -np.log(1e-6 + 1.0 / np.clip(opa_, 1e-6, 1.0) - 1.0).astype(np.float32)
log_scale = np.full(N, np.log(base_scale / 3.), dtype=np.float32)

rot = np.zeros((N, 4), dtype=np.float32)
rot[:, 0] = 1.0

ply_dtype = np.dtype([
    ("x", "<f4"), ("y", "<f4"), ("z", "<f4"),
    ("f_dc_0", "<f4"), ("f_dc_1", "<f4"), ("f_dc_2", "<f4"), ("opacity", "<f4"),
    ("scale_0", "<f4"), ("scale_1", "<f4"), ("scale_2", "<f4"),
    ("rot_0", "<f4"), ("rot_1", "<f4"), ("rot_2", "<f4"), ("rot_3", "<f4"),
])

rec = np.empty(N, dtype=ply_dtype)
rec["x"], rec["y"], rec["z"] = positions[:, 0], positions[:, 1], positions[:, 2]
rec["f_dc_0"], rec["f_dc_1"], rec["f_dc_2"] = shs[:, 0], shs[:, 1], shs[:, 2]
rec["opacity"] = opacities
rec["scale_0"], rec["scale_1"], rec["scale_2"] = log_scale, log_scale, log_scale
rec["rot_0"], rec["rot_1"], rec["rot_2"], rec["rot_3"] = rot[:, 0], rot[:, 1], rot[:, 2], rot[:, 3]

# 写出二进制little-endian PLY：header + 一次性二进制块
with open(output_ply, "wb") as f:
    header = []
    header.append("ply\n")
    header.append("format binary_little_endian 1.0\n")
    header.append(f"element vertex {N}\n")
    header.append("property float x\n")
    header.append("property float y\n")
    header.append("property float z\n")
    header.append("property float f_dc_0\n")
    header.append("property float f_dc_1\n")
    header.append("property float f_dc_2\n")
    header.append("property float opacity\n")
    header.append("property float scale_0\n")
    header.append("property float scale_1\n")
    header.append("property float scale_2\n")
    header.append("property float rot_0\n")
    header.append("property float rot_1\n")
    header.append("property float rot_2\n")
    header.append("property float rot_3\n")
    header.append("end_header\n")
    f.write("".join(header).encode("ascii"))

    # 结构化数组已为小端，直接写出
    rec.tofile(f)

print(f"Saved binary PLY to: {output_ply}")

Saved binary PLY to: /home/tatp/Desktop/points_thresh_supersplat.ply


In [11]:
grids = np.load("/home/tatp/Desktop/test_grids.npy")
print(grids.shape, grids.dtype)

(1357959, 4) float32


In [ ]:
import numpy as np
import open3d as o3d

grids = np.load("/home/tatp/Desktop/grids.npy")  # (N,4) x,y,z,density
density_threshold = 1.0
voxel = 0.02  # 体素下采样，越大点越少，渲染越快

mask = grids[:, 3] > density_threshold
pts = grids[mask, :3].astype(np.float32)

# 可选：再做随机下采样避免过多点
max_points = 5_000_000
if pts.shape[0] > max_points:
    idx = np.random.choice(pts.shape[0], max_points, replace=False)
    pts = pts[idx]

pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
pcd.paint_uniform_color([0.0, 0.0, 1.0])  # 蓝色
pcd = pcd.voxel_down_sample(voxel_size=voxel)

o3d.visualization.draw_geometries([pcd])

In [ ]:
import taichi as ti
import numpy as np

ti.init(arch=ti.gpu)
grids = np.load("/home/tatp/Desktop/grids.npy")
mask = grids[:, 3] > 1.0
pts = grids[mask, :3].astype(np.float32)

# 可选再抽样
max_points = 5_000_000
if pts.shape[0] > max_points:
    idx = np.random.choice(pts.shape[0], max_points, replace=False)
    pts = pts[idx]

pos = ti.Vector.field(3, dtype=ti.f32, shape=pts.shape[0])
pos.from_numpy(pts)

window = ti.ui.Window("PointCloud", (1280, 720))
canvas = window.get_canvas()
scene = ti.ui.Scene()
camera = ti.ui.Camera()
camera.position(0, 0, 3)
camera.lookat(0, 0, 0)

while window.running:
    camera.track_user_inputs(window, movement_speed=0.5, hold_key=ti.ui.RMB)
    scene.set_camera(camera)
    scene.ambient_light((0.8, 0.8, 0.8))
    scene.particles(pos, radius=0.002, color=(0.0, 0.0, 1.0))
    canvas.scene(scene)
    window.show()